# Linear Transformations
---
**Name:** Jonh Alexis Buot <br>
**Date:** December 2023 <br>
**Course:** CS3101N <br>
**Task:** Assignment - Linear Transformations

---

In [673]:
import random
import numpy as np

# Code Challenges

**1. Develop a python function from scratch that will find the determinants of any $n$ x $n$ matrix.**

In [674]:
def mat_det(A):
# =============================================================================
#   Calculate the determinant of a square matrix A.
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."
    
    n = len(A)
    if   n == 0: return 1.
    elif n == 1: return A[0][0]
    elif n == 2: return A[0][0] * A[1][1] - A[0][1] * A[1][0] 

    det = 0.
    for col, cofactor in enumerate(A[0]):
        minor = [[A[i][j] for j in range(n) if j != col] for i in range(1, n)]
        det += (-1) ** col * cofactor * mat_det(minor)

    return det

**2. Develop a python function from scratch that will find both the eigenvectors and eigenvalues of any $n$ x $n$ matrix.**

In [714]:
def mat_eig(A, max_iterations = 10000):
# =============================================================================
#   Calculate the eigenvectors and eigenvalues of a square matrix A.
#   
#   Note: This uses power method with hotelling deflation.
#   https://services.math.duke.edu/~jtwong/math361-2019/lectures/Lec10eigenvalues.pdf
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."

    n = len(A) 
    if n == 0: return [], []
    if n == 1: return [A[0][0]], [[1.]]

    eigenvectors = []
    eigenvalues = []

    for _ in range(n):
        # Generate random guess for eigenvector with ||eigvec|| = 1
        eigvec = [random.random() for _ in range(n)]
        norm_eigvec = sum(a ** 2 for a in eigvec) ** 0.5
        eigvec = [a / norm_eigvec for a in eigvec]

        # Power Method
        for _ in range(max_iterations):
            tf_eigvec = [sum(a * b for a, b in zip(row, eigvec)) for row in A]
            
            norm_tf = sum(a ** 2 for a in tf_eigvec) ** 0.5
            eigvec = [a / norm_tf for a in tf_eigvec]
            
        eigval = sum(a * b for a, b in zip(tf_eigvec, eigvec))
        eigenvalues.append(eigval)
        eigenvectors.append(eigvec)

        # Deflate Matrix
        eigenval_outer_product = [[eigval * a * b / eigval for b in eigvec] for a in tf_eigvec]
        A = [[A[i][j] - eigenval_outer_product[i][j] for j in range(n)] for i in range(n)]

    return eigenvalues, [list(row) for row in zip(*eigenvectors)]

**3. Test your functions from a randomly generated $n$ x $n$ matrix.**

- Testing and comparing my determinant implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [679]:
A = np.random.random((8, 8)).astype("float64") * 5

my_result = mat_det(A.tolist())
np_result = np.linalg.det(A)

print("-- A --", A, sep="\n")
print("\n-- My det(A) --", my_result, sep="\n")
print("\n-- Numpy det(A) --", np_result, sep="\n")
print("\nAre my results equal to numpy?", "Yes." if np.isclose(my_result, np_result) else "No.")

-- A --
[[1.98807091 2.40997936 1.75637492 4.39015175 2.18998783 2.93207122
  4.04727582 2.81296037]
 [3.12364052 0.49919794 3.81875341 1.98708201 2.64102244 0.0851378
  1.15345157 1.71538701]
 [1.2565729  2.76963438 2.17961284 2.85986999 3.78769557 1.41499935
  0.68529863 4.40412522]
 [0.3164824  0.38951355 0.05162677 4.82031128 3.67667059 1.0963566
  4.61590761 2.87741637]
 [2.8022465  0.86121105 2.68352865 4.98150003 2.19348394 1.65629326
  0.93531264 0.10097983]
 [3.02681458 0.24030133 4.79512195 0.79668573 3.21064113 0.65536868
  2.50885955 4.14700326]
 [1.1140123  0.05163665 1.89508189 3.7598244  1.93253434 1.7822086
  4.75691959 3.25257158]
 [3.48592547 1.51564648 2.67646586 2.05514124 1.21059168 4.58517945
  0.75908825 4.21362902]]

-- My det(A) --
2942.5506423478587

-- Numpy det(A) --
2942.550642347845

Are my results equal to numpy? Yes.


- Testing and comparing my eigenvalue and eigenvector algorithm implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [872]:
A = np.random.random((3, 3)).astype("float64") * 5

np_ev, np_evl = np.linalg.eig(A)
m_ev, m_evl = mat_eig(A.tolist())
m_ev, m_evl = np.array(m_ev), np.array(m_evl)

print("-- A --", A, sep="\n")
print("\n-- My Eigenvalues and Eigenvectors --", m_ev, m_evl, sep="\n")
print("\n-- Numpy Eigenvalues and Eigenvectors --", np_ev, np_evl, sep="\n")

print("\n\n", "Are the differences of eigenvalues close?", "Yes." if np.allclose(m_ev, np_ev) else "No.")

cosine_similarity = np.array([abs(npe.dot(me) / (np.linalg.norm(me) * np.linalg.norm(npe))) for npe, me in zip(np_evl.T, m_evl.T)]).T
print("\n-- Cosine Similarity of Eigenvalues --", cosine_similarity, sep="\n")

-- A --
[[4.20677299 0.22257174 2.40438209]
 [1.61750839 4.17822603 3.4276382 ]
 [1.70149125 0.04315884 1.48650216]]

-- My Eigenvalues and Eigenvectors --
[5.68586751 3.74710003 0.43853365]
[[ 0.432322    0.90181502 -0.92085443]
 [ 0.88269675  0.13282581  0.04043162]
 [ 0.18423933  0.41120186 -0.38780459]]

-- Numpy Eigenvalues and Eigenvectors --
[5.68586751 3.74710003 0.43853365]
[[ 0.432322    0.11720146 -0.44773816]
 [ 0.88269675 -0.99068727 -0.49117473]
 [ 0.18423933  0.0693004   0.74717998]]


 Are the differences of eigenvalues close? Yes.

-- Cosine Similarity of Eigenvalues --
[1.         0.00260165 0.10268285]
